In [161]:
from DiSuQ.Torch import models as TorchModels
from DiSuQ.Torch.optimization import OrderingOptimization
from DiSuQ.Torch.optimization import lossAnharmonicity,lossTransitionFlatness

from DiSuQ.Kerman import models as KermanModels
from scipy.optimize import minimize

from numpy import polyfit,log10 as log,exp,prod
from torch import tensor
from numpy import arange,linspace,var,logspace
from DiSuQ.utils import plotCompare
from torch import set_num_threads
from time import perf_counter
from memory_profiler import memory_usage
import torch,pandas
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 
set_num_threads(20)

In [2]:
device = torch.device(0)

In [3]:
def torchProfiling(N,El=10.,Ec=100.,Ej=50):
    basis = {'O':[N],'I':[],'J':[]}; rep = 'K'
    circuit = TorchModels.fluxonium(basis,El=El,Ec=Ec,Ej=Ej,sparse=False)
    flux_range = tensor(linspace(0,1,n_flux,endpoint=True))
    
    flux_profile = [{'I':flux} for flux in flux_range]
    optim = OrderingOptimization(circuit,representation=rep)
    dLogs,dParams,dCircuit = optim.optimization(lossTransitionFlatness,flux_profile,iterations=n_iter,device=device)
    #print(dLogs)
        
def numpyProfiling(N,El=10.,Ec=100.,Ej=50):
    basis = {'O':[N],'I':[],'J':[]}; rep = 'K'
    flux_range = linspace(0,1,n_flux,endpoint=True)
    
    flux_range = [[flux] for flux in flux_range]
    loss = lossBuilder(basis,flux_range)
    init_pos = El,Ej,Ec
    bounds = (0.1,500),(0.1,1500),(0.1,1500.)
    res = minimize(loss, init_pos, method = 'L-BFGS-B', bounds = bounds, options = {'maxiter': n_iter})
    #print(res)

def lossFlatness(E0,E1,E2):
    loss = var(E1-E0)
    loss += var(E2-E1)
    return loss

def lossBuilder(basis,flux_range):
    def loss(circuit):
        El,Ej,Ec = circuit
        circuit = KermanModels.fluxonium(basis,El=El,Ec=Ec,Ej=Ej)
        H_LC = circuit.kermanHamiltonianLC()
        H_J = circuit.kermanHamiltonianJosephson
        E0,(E1,E2) = circuit.spectrumManifold('I',flux_range,H_LC,H_J,[1,2])
        loss = lossFlatness(E0,E1+E0,E2+E0)
        return loss
    return loss

In [54]:
n_iter = 8
n_bases = 8
n_flux = 8

In [56]:
dTor = []
bases = logspace(1,4,n_bases,dtype=int)
print(bases)
arguments = {'El':10.,'Ec':100.,'Ej':50.}
for N in bases:
    print(N)
    start = perf_counter()
    mem = memory_usage((torchProfiling,(N,)))
    time = perf_counter()-start
    mem = max(mem)-min(mem)
    dTor.append({'lib':'torch','mem':mem,'time':time,'iter':n_iter,'hilbert':N})
    

#     start = perf_counter()
#     mem = memory_usage((numpyProfiling,(N,)))
#     time = perf_counter()-start
#     mem = max(mem)-min(mem)
#     dLog.append({'lib':'numpy','mem':mem,'time':time,'iter':n_iter,'hilbert':N})

[   10    26    71   193   517  1389  3727 10000]
10
26
71
193
517
1389
3727
10000


Process MemTimer-26:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/chishti/env/lib/python3.11/site-packages/memory_profiler.py", line 262, in run
    stop = self.pipe.poll(self.interval)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/multiprocessing/connection.py", line 440, in _poll
    r = wait([self], timeout)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/multiprocessing/connection.py", line 947, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

# Iteration time estimation

In [57]:
dTor = pandas.DataFrame(dTor)

In [178]:
dTor

,lib,mem,time,iter,hilbert
0,torch,0.062500,0.914513,8,10
1,torch,0.019531,1.113356,8,26
2,torch,0.000000,1.987668,8,71
3,torch,0.460938,8.848764,8,193
4,torch,1.421875,57.229733,8,517
5,torch,0.050781,687.236434,8,1389
6,torch,21408.015625,7831.367500,8,3727


In [106]:
x = log(dTor['hilbert'])
y = log(dTor['time'])

In [107]:
a,b,c = polyfit(x,y,2)

In [108]:
timeOpt = lambda h: 10**(a*log(h)**2+b*log(h)+c)
# time = m * log(base) + c

In [110]:
plotCompare(log(dTor['hilbert']),{'time':log(dTor['time']),'estimate':log(timeOpt(dTor['hilbert']))})

In [111]:
timeOpt(3700)

8989.18473852394

# Workload Calculation

## Discovery
* assuming timeOpt is without I/O bottleneck

### ZeroPi

In [228]:
parameters = 4
n_flux = 10
n_Hilbert = 20
n_grid = 10
runs = n_grid**parameters 
print('runs:',runs)

runs: 10000


In [229]:
time = timeDiagDense(n_Hilbert**3)
time = time/48/60
print('wall time:',time)
time *= runs * n_flux
print('core-hour(M):',time*48/1e6)

wall time: 0.08093023081340583
core-hour(M): 0.388465107904348


### Fluxonium

In [147]:
parameters = 3
n_flux = 10
n_Hilbert = 1000
n_grid = 20

print('runs:',runs)

In [146]:
time = n_grid**parameters * n_flux * timeOpt(n_Hilbert)
time/60/1e6

0.06664610174512872

### Prismon

In [218]:
parameters = 4
n_flux = 1
n_Hilbert = 7
n_grid = 10
runs = n_grid**parameters # runs
print('runs:',runs)

runs: 10000


In [219]:
time = timeOpt(n_Hilbert**5)
time = time/48/60
print('wall time:',time)
time *= runs * n_flux
print('core-hour(M):',time*48/1e6)

wall time: 1.3971820994829711
core-hour(M): 0.6706474077518261


## Basis Convergence

#### Diagonalization Time estimation

In [125]:
dGPUCPU = pandas.read_csv('../Computation/sparse-dense-gpu.csv')
dGPUCPU1 = pandas.read_csv('../Computation/sparse-dense-gpu-small.csv')
dGPUCPU = pandas.concat([dGPUCPU1,dGPUCPU])
dGPUCPU = dGPUCPU[dGPUCPU['rep']=='dense']
dGPUCPU = dGPUCPU[dGPUCPU['threads']==64]

In [126]:
x = log(dGPUCPU['hilbert'])
y = log(dGPUCPU['time'])

In [128]:
a,b,c = polyfit(x,y,2)
timeDiagDense = lambda h: 10**(a*log(h)**2+b*log(h)+c)

In [129]:
plotCompare(log(dGPUCPU['hilbert']),{'time':log(dGPUCPU['time']),'estimate':log(timeDiagDense(dGPUCPU['hilbert']))})

### Fluxonium

In [198]:
parameters = 3
n_flux = 100 # step/run
n_grid = 10 # 
runs = n_grid**parameters # runs
print('runs:',runs)

runs: 1000


In [201]:
time = [timeDiagDense(H) for H in linspace(1000,5000,5)]
time = sum(time)/48/60
print('wall time:',time)
time *= runs * n_flux
print('core-hour(M):',time*48/1e6)

wall time: 0.03241141226521801
core-hour(M): 0.15557477887304647


### Prismon

In [182]:
parameters = 4
n_flux = 10
n_Hilbert = 100
n_grid = 8

In [183]:
time = [timeDiagDense(H**5) for H in linspace(5,8,5)]
time = sum(time)
print(time)
time *= n_grid**parameters * n_flux
time/60/1e6

3.5381820542541188e+90


2.4153989490374782e+87

# Computer Comparison

In [7]:
dLog = pandas.read_csv('fluxonium-optimization-gpu.csv')

In [11]:
gpu = dLog[dLog['lib']=='torch']['time'].iloc[:-1]
cpu = dLog[dLog['lib']=='numpy']['time'].iloc[:-1]

In [14]:
plotCompare(bases,{'GPU':gpu,'CPU':cpu},None,'basis size','time(sec)',size=(600,600))